<a href="https://colab.research.google.com/github/hanhanwu/Hanhan_COLAB_Experiemnts/blob/master/GenAI_Practice/Langwatch/dspy_prompt_optimization_online_dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangWatch DSPy Visualizer

This notebook shows an example of a simple DSPy optimization process integrated with LangWatch for training visualization and debugging.

In [1]:
%%capture --no-stderr
!pip install --upgrade nbformat
%pip install -U --quiet dspy langwatch

## Preparing the LLM

In [3]:
import os
from getpass import getpass
import dspy
from google.colab import userdata


# OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
# llm = dspy.LM("openai/gpt-4.1-nano", api_key=OPENAI_API_KEY)

GOOGLE_AI_API_KEY = userdata.get('GOOGLE_AI_API_KEY')
llm = dspy.LM("gemini/gemini-2.0-flash", api_key=GOOGLE_AI_API_KEY)
print("LLM test response:", llm("Where's Silicon Valley?"))

# the retrieval model
colbertv2_wiki17_abstracts = dspy.ColBERTv2(
    url="http://20.102.90.50:2017/wiki17_abstracts"
)
dspy.settings.configure(lm=llm, rm=colbertv2_wiki17_abstracts)

LLM test response: ['Silicon Valley is located in the southern part of the San Francisco Bay Area in **Northern California, United States**.\n']


## Preparing the Dataset

In [4]:
from dspy.datasets import HotPotQA


dataset = HotPotQA(train_seed=1, train_size=32, eval_seed=2025, dev_size=50, test_size=0)
trainset = [x.with_inputs('question') for x in dataset.train]
devset = [x.with_inputs('question') for x in dataset.dev]

print(len(trainset), len(devset))
print(trainset[0])
print(devset[0])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.19k [00:00<?, ?B/s]

hotpot_qa.py:   0%|          | 0.00/6.42k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/90447 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/7405 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7405 [00:00<?, ? examples/s]

32 50
Example({'question': 'At My Window was released by which American singer-songwriter?', 'answer': 'John Townes Van Zandt'}) (input_keys={'question'})
Example({'question': 'Pehchaan: The Face of Truth stars Vinod Khanna, Rati Agnihotri and which Indian actress, producer, and former model who also produced the film?', 'answer': 'Raveena Tandon', 'gold_titles': {'Pehchaan: The Face of Truth', 'Raveena Tandon'}}) (input_keys={'question'})


## Defining the model

In [5]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""
    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")


class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()
        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)

    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)


dev_example = devset[12]
print(f"[Devset] Question: {dev_example.question}")
print(f"[Devset] Answer: {dev_example.answer}")
print(f"[Devset] Relevant Wikipedia Titles: {dev_example.gold_titles}")
print()

generate_answer = RAG()
pred = generate_answer(question=dev_example.question)
print(f"[Prediction] Question: {dev_example.question}")
print(f"[Prediction] Predicted Answer: {pred.answer}")

[Devset] Question: Twelve Inches is a compilation album by which 1980s British band?
[Devset] Answer: Frankie Goes to Hollywood
[Devset] Relevant Wikipedia Titles: {'Twelve Inches', 'Frankie Goes to Hollywood'}

[Prediction] Question: Twelve Inches is a compilation album by which 1980s British band?
[Prediction] Predicted Answer: Bananarama


## Login to LangWatch

* Run LangWatch online dashboard

In [6]:
import langwatch


langwatch.endpoint = "https://app.langwatch.ai"
langwatch.setup(api_key=userdata.get('LANGWATCH_API_KEY'))
langwatch.login()

2025-05-22 02:50:50,032 - langwatch.client - INFO - Registering atexit handler to flush tracer provider on exit


INFO:langwatch.client:Registering atexit handler to flush tracer provider on exit


LangWatch API key is already set, if you want to login again, please call as langwatch.login(relogin=True)


## Start Training Session!

* How does Langwatch use DsPy
  * Langwatch's DsPy init: https://github.com/langwatch/langwatch/blob/main/python-sdk/src/langwatch/dspy.py#L151-L205
  * DsPy Optimizer `MIPROv2`: https://github.com/stanfordnlp/dspy/blob/main/docs/docs/api/optimizers/MIPROv2.md
    * A list of DsPy optimizers: https://github.com/stanfordnlp/dspy/tree/main/docs/docs/api/optimizers

In [7]:
from dspy.teleprompt import MIPROv2
import dspy.evaluate


# Define our metric validation
def validate_context_and_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_EM and answer_PM

# Set up a MIPROv2 optimizer, which will compile our RAG program.
optimizer = MIPROv2(metric=validate_context_and_answer, prompt_model=llm,
                    task_model=llm, num_candidates=2, init_temperature=0.7,
                    auto=None)

# Initialize langwatch for this run, to track the optimizer compilation
langwatch.dspy.init(experiment="hanhan_exp1", optimizer=optimizer)

# Compile
compiled_rag = optimizer.compile( RAG(),
    trainset=trainset,
    num_trials=10,
    max_bootstrapped_demos=3,
    max_labeled_demos=5,
    minibatch_size=10,
    requires_permission_to_run=False
)

2025/05/22 02:50:53 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/05/22 02:50:53 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/05/22 02:50:53 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=2 sets of demonstrations...
2025/05/22 02:50:53 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 2: PROPOSE INSTRUCTION CANDIDATES <==
2025/05/22 02:50:53 INFO dspy.teleprompt.mipro_optimizer_v2: We will use the few-shot examples from the previous step, a generated dataset summary, a summary of the program code, and a randomly selected prompting tip to propose instructions.



[LangWatch] Experiment initialized, run_id: nifty-buzzard-of-judgment
[LangWatch] Open https://app.langwatch.ai/my-garden-vZCaox/experiments/hanhan-exp1?runIds=nifty-buzzard-of-judgment to track your DSPy training session live

Bootstrapping set 1/2
Bootstrapping set 2/2


2025/05/22 02:50:56 INFO dspy.teleprompt.mipro_optimizer_v2: 
Proposing N=2 instructions...

2025/05/22 02:51:06 INFO dspy.teleprompt.mipro_optimizer_v2: Proposed Instructions for Predictor 0:

2025/05/22 02:51:06 INFO dspy.teleprompt.mipro_optimizer_v2: 0: Answer questions with short factoid answers.

2025/05/22 02:51:06 INFO dspy.teleprompt.mipro_optimizer_v2: 1: Given the provided context, answer the question with a concise, fact-based response. First, reason step-by-step to understand the question and identify the relevant information in the context. Then, provide the answer directly, using as few words as possible.

2025/05/22 02:51:06 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/05/22 02:51:06 INFO dspy.teleprompt.mipro_optimizer_v2: ==> STEP 3: FINDING OPTIMAL PROMPT PARAMETERS <==
2025/05/22 02:51:06 INFO dspy.teleprompt.mipro_optimizer_v2: We will evaluate the program over a series of trials with different combinations of instructions and few-shot examples to find the optim

Average Metric: 4.00 / 6 (66.7%):  20%|██        | 5/25 [00:02<00:05,  3.35it/s]

2025/05/22 02:51:12 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/05/22 02:51:12 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/05/22 02:51:13 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/05/22 02:51:14 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/05/22 02:51:14 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/05/22 02:51:15 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/05/22 02:51:15 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/05/22 02:51:15 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/05/22 02:51

...

Average Metric: 6.00 / 16 (37.5%):  92%|█████████▏| 23/25 [00:18<00:01,  1.12it/s]

2025/05/22 02:54:07 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/05/22 02:54:12 ERROR dspy.utils.parallelizer: Error for Example({'question': 'Which movie was released first, Son of Flubber or Davy Crockett, King of the Wild Frontier?', 'answer': 'Davy Crockett, King of the Wild Frontier'}) (input_keys={'question'}): Both structured output format and JSON mode failed. Please choose a model that supports `response_format` argument. Original error: litellm.RateLimitError: litellm.RateLimitError: VertexAIException - {
  "error": {
    "code": 429,
    "message": "You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.",
    "status": "RESOURCE_EXHAUSTED",
    "details": [
      {
        "@type": "type.googleapis.com/google.rpc.QuotaFailure",
        "violations": [
          {
            "quotaMetric": "genera

Average Metric: 6.00 / 16 (37.5%): 100%|██████████| 25/25 [00:25<00:00,  1.01s/it]

2025/05/22 02:54:12 INFO dspy.evaluate.evaluate: Average Metric: 6.0 / 25 (24.0%)


2025/05/22 02:54:13 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [32.0, 36.0, 24.0]
2025/05/22 02:54:13 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 36.0
2025/05/22 02:54:13 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/05/22 02:54:13 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/05/22 02:54:13 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 36.0!


In [8]:
compiled_rag

generate_answer.predict = Predict(StringSignature(context, question -> reasoning, answer
    instructions='Answer questions with short factoid answers.'
    context = Field(annotation=str required=True json_schema_extra={'desc': 'may contain relevant facts', '__dspy_field_type': 'input', 'prefix': 'Context:'})
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    reasoning = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${reasoning}', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'desc': 'often between 1 and 5 words', '__dspy_field_type': 'output', 'prefix': 'Answer:'})
))

In [9]:
compiled_rag.save("optimized_model.json")